In [75]:
% setup environment
% set path to equilibrium solver
txt=pwd;
n=length(txt);
rootpath=txt; rootpath=txt(1:n-19);
PHREEQCpath=[rootpath,'/runPHREEQC'];
addpath(PHREEQCpath);
JEQUILIBpath=[rootpath,'/JEQUILIB'];
addpath(JEQUILIBpath);
% clear data and number format
clear; format short e
% turn off warnings
warning off
graphics_toolkit ("notebook"); 
%graphics_toolkit ("plotly"); 
%graphics_toolkit("gnuplot")
%graphics_toolkit("fltk")

In [76]:
% MODEL USING TABLEAU (different gradients or log versus linear variables)
%graphics_toolkit("notebook")
%model with just inorganic speciation
%geochemical model
% input the constants and solution chemistry

flag1=1; %flag1 for concentration based search vector (1) or for log based search vector (2).
flag2=1; %flag2 for numerical derivatives (2) or analtyical derivatives (1)
database=[]; 
flag3=0; % flag3 1 for show warnings. 0 no warnings
flag4=0; %if=1 solve tableau one line at a time to get a good initial guess (no solids). really challenging probs only
flag5=0; %if=1 use stored initial guess for including solids

AgT=1.003e-5; ClT=1.003e-5; pH=7; %for phreeqc would be 1.003e-5 because inputs should be as molality concentrations, using 0.997 kg/L as density at 25 °C

inorganicTOTALS=[AgT ClT];
inorganicTOTALS(inorganicTOTALS==0)=1e-16; % get rid of zero values, b/c div by zero error
TOTALS=[inorganicTOTALS]; %TOTALS=inorganicTOTALS;
pH=pH; pe=20.75-pH; 
tic;
[Ag,Cl,AgCls,MASSERR]=AgCltableaumorecomplex(pH,pe,TOTALS',flag1,flag2,flag3,flag4,flag5);
tableautime=toc;
Agmasserror=MASSERR(1); Clmasserror=MASSERR(2);

In [77]:
% MODEL USING PHREEQC

minerals=[{'AgCls'}]; totalvector=[AgT; ClT; ClT]; totalnames=[{'Ag'}; {'Cl'}; {'Na'}]; 
speciesexport=[{'Ag+'}; {'Cl-'}; {'AgCl'}; {'AgCl2-'}; {'AgCl3-2'}; {'AgCl4-3'}];
%database=['llnl_nosolubleAgCl.dat']; 
acid=['NaOH']; show=1; T=25;
%database=['AgCl.dat']; 
%database=['AgClwithcomplexes.dat'];
database=['AgClwithcomplexes_ideal.dat']; 

tic
[solutionspeciesconcs, speciesnames, solidconcs, solidnames]=runPHREEQCv2noHA(T,pH,pe,totalnames,totalvector,minerals,speciesexport,database,show,acid);
PHREEQCtime=toc;

AgPHREEQC=solutionspeciesconcs(1); 
ClPHREEQC=solutionspeciesconcs(2); 
AgClPHREEQC=solutionspeciesconcs(3); 
AgCl2PHREEQC=solutionspeciesconcs(4); 
AgCl3PHREEQC=solutionspeciesconcs(5); 
AgCl4PHREEQC=solutionspeciesconcs(6); 
AgClsPHREEQC=solidconcs(1);

AgmasserrorPHREEQC=AgT-AgPHREEQC-AgClsPHREEQC-AgClPHREEQC-AgCl2PHREEQC-AgCl3PHREEQC-AgCl4PHREEQC; 
ClmasserrorPHREEQC=ClT-ClPHREEQC-AgClsPHREEQC-AgClPHREEQC-2*AgCl2PHREEQC-3*AgCl3PHREEQC-4*AgCl4PHREEQC; 

In [78]:
% exact model

Ksp=10^-9.7453;

% check SI

IAP=AgT*ClT;

Agexact=AgT; Clexact=ClT; AgClsexact=0;

if IAP>=Ksp
a=1; b=ClT-AgT; c=-Ksp;
t=roots([a b c]); t=t(t>0); t=real(t(imag(t)==0));
Agexact=t; AgClsexact=AgT-Ag; Clexact=ClT-AgClsexact;
end


In [79]:
% compare models

Agcompare=[Ag AgPHREEQC*0.997 Agexact]
Clcompare=[Cl ClPHREEQC*0.997 Clexact]
AgClcompare=[AgCls AgClsPHREEQC*0.997 AgClsexact]
Agmasserrorcompare=[Agmasserror AgmasserrorPHREEQC*0.997]
Clmasserrorcompare=[Clmasserror ClmasserrorPHREEQC*0.997]
timecompare=[tableautime PHREEQCtime]

Agcompare,1,2,3
1,9.83799e-06,9.80096e-06,1.003e-05


Clcompare,1,2,3
1,9.8378e-06,9.80076e-06,1.003e-05


AgClcompare,1,2,3
1,0,0,0


Agmasserrorcompare,1,2
1,8.44499e-14,-2.1639e-19


Clmasserrorcompare,1,2
1,3.378e-14,2.92364e-19


timecompare,1,2
1,0.218823,0.02088


In [80]:
%% Repeat PHREEQC 50 times and report times

% Number of runs
nRuns = 50;
% Vector to store the time of each run
timesPHREEQC = zeros(nRuns, 1);

% Declare variables where will save the result of the last run
AgPHREEQC_last   = 0;
ClPHREEQC_last   = 0;
AgClsPHREEQC_last= 0;
AgMassErr_last   = 0;
ClMassErr_last   = 0;

for i = 1:nRuns
    tic;
    [solutionspeciesconcs, speciesnames, solidconcs, solidnames] = ...
        runPHREEQCv2noHA(T, pH, pe, totalnames, totalvector, ...
                         minerals, speciesexport, database, show, acid);
    timesPHREEQC(i) = toc;
    
    % Each iteration results
    AgPHREEQC = solutionspeciesconcs(1);
    ClPHREEQC = solutionspeciesconcs(2);
    AgClsPHREEQC = solidconcs(1);

    % Calculation of mass errors for this run
    AgmasserrorPHREEQC=AgT-AgPHREEQC-AgClsPHREEQC-AgClPHREEQC-AgCl2PHREEQC-AgCl3PHREEQC-AgCl4PHREEQC; 
    ClmasserrorPHREEQC=ClT-ClPHREEQC-AgClsPHREEQC-AgClPHREEQC-2*AgCl2PHREEQC-3*AgCl3PHREEQC-4*AgCl4PHREEQC; 

    % Save as "last" run (overwrite it in each loop)
    AgPHREEQC_last    = AgPHREEQC;
    ClPHREEQC_last    = ClPHREEQC;
    AgClsPHREEQC_last = AgClsPHREEQC;
    AgMassErr_last    = AgmasserrorPHREEQC;
    ClMassErr_last    = ClmasserrorPHREEQC;
end

% Mean and standard deviation of the 50 times
meanTimePHREEQC = mean(timesPHREEQC);
stdTimePHREEQC  = std(timesPHREEQC);

% Display results
fprintf('\n========== PHREEQC executed %d veces ==========\n', nRuns);
fprintf('Last run:\n');
fprintf('  [Ag(aq)]    = %.6e M\n', AgPHREEQC_last*0.997);
fprintf('  [Cl(aq)]    = %.6e M\n', ClPHREEQC_last*0.997);
fprintf('  [AgCl(s)]   = %.6e M\n', AgClsPHREEQC_last*0.997);
fprintf('  Error mass Ag = %.4e\n', AgMassErr_last*0.997);
fprintf('  Error mass Cl = %.4e\n', ClMassErr_last*0.997);
fprintf('-----------------------------------------------\n');
fprintf('Mean time in 50 runs (ms) = %.6f\n', meanTimePHREEQC*1000);
fprintf('Standard Deviation (ms)            = %.6f\n', stdTimePHREEQC*1000);
fprintf('===============================================\n');


========== PHREEQC executed 50 veces ==========
Last run:
  [Ag(aq)]    = 9.800958e-06 M
  [Cl(aq)]    = 9.800758e-06 M
  [AgCl(s)]   = 0.000000e+00 M
  Error mass Ag = -2.1639e-19
  Error mass Cl = 2.9236e-19
-----------------------------------------------
Mean time in 50 runs (ms) = 21.141615
Standard Deviation (ms)            = 4.992936


In [81]:
% Tableau std and mean times

nRuns = 50;                % número de corridas
timesTableau = zeros(nRuns,1);

% Variables to save the solution of the last run
Ag_last   = 0;
Cl_last   = 0;
AgCls_last= 0;
massErr_last = [0 0];

for i = 1:nRuns
    tStart = tic;
    [Ag, Cl, AgCls, MASSERR] = AgCltableau(pH, pe, TOTALS', ...
                                          flag1, flag2, flag3, ...
                                          flag4, flag5);
    timesTableau(i) = toc(tStart);

    % saving the results of this run
    Ag_last    = Ag;
    Cl_last    = Cl;
    AgCls_last = AgCls;
    massErr_last = MASSERR;
end

% Calculation of mean and standard deviation of times
meanTimeTableau = mean(timesTableau);
stdTimeTableau  = std(timesTableau);

% Final information of the last run
Agmasserr_last = massErr_last(1);
Clmasserr_last = massErr_last(2);

% Display results
fprintf('\n========== Tableau Scott (AgCltableau) executed %d veces ==========\n', nRuns);
fprintf('Last run:\n');
fprintf('  [Ag(aq)]    = %.6e M\n', Ag_last);
fprintf('  [Cl(aq)]    = %.6e M\n', Cl_last);
fprintf('  [AgCl(s)]   = %.6e M\n', AgCls_last);
fprintf('  Error mass Ag = %.4e\n', Agmasserr_last);
fprintf('  Error masa Cl = %.4e\n', Clmasserr_last);
fprintf('--------------------------------------------------------------\n');
fprintf('Mean time in %d runs (ms) = %.6f\n', nRuns, meanTimeTableau*1000);
fprintf('Standard deviation (ms)            = %.6f\n', stdTimeTableau*1000);
fprintf('==============================================================\n');


========== Tableau Scott (AgCltableau) executed 50 veces ==========
Last run:
  [Ag(aq)]    = 1.003000e-05 M
  [Cl(aq)]    = 1.003000e-05 M
  [AgCl(s)]   = 0.000000e+00 M
  Error mass Ag = 5.0670e-14
  Error masa Cl = 1.5201e-13
--------------------------------------------------------------
Mean time in 50 runs (ms) = 162.858243
Standard deviation (ms)            = 1.612344


In [82]:
%% Snippet to parse the “End of Run after ... Seconds.” from out.txt and 
%% compute the mean and std of the internal times, skipping any NaNs.

nRuns = 50;
intTimes = zeros(nRuns, 1);

for i = 1:nRuns
    % (1) Run PHREEQC or your call that produces out.txt

    % (2) Parse the line "End of Run after X.XXXX Seconds." from out.txt
    [status, outStr] = system("grep 'End of Run after' out.txt");
    % outStr might look like "End of Run after X.XXXX Seconds."

    expr = 'after\s+([0-9]*\.[0-9]+|[0-9]+)\s+Seconds';
    tokens = regexp(outStr, expr, 'tokens', 'once');

    if ~isempty(tokens)
        intTimes(i) = str2double(tokens{1});
    else
        % If we don't find the line, store NaN
        intTimes(i) = NaN;
    end
end

% Filter out any NaN entries
validIdx = ~isnan(intTimes);
validTimes = intTimes(validIdx);

% Compute mean and std for the valid subset
meanIntTime = mean(validTimes);
stdIntTime  = std(validTimes);

fprintf('\nInternal PHREEQC time across %d runs:\n', nRuns);
fprintf('  Mean (ms) = %.6f\n', meanIntTime*1000);
fprintf('  Std  (ms) = %.6f\n\n', stdIntTime*1000);



Internal PHREEQC time across 50 runs:
  Mean (ms) = 3.286000
  Std  (ms) = 0.000000

